In [19]:
import mysql.connector
import requests
import urllib
import pandas as pd
import json

In [20]:
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="vuQDE37idjRX",
  database="nfl"
)
cursor = mydb.cursor()

In [21]:
cursor.execute('SELECT * FROM stadiums')


In [22]:
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[i[0] for i in cursor.description])

In [23]:


# USGS Elevation Point Query Service
#url = r'https://nationalmap.gov/epqs/pqs.php?'
#new 2023:
url = r'https://epqs.nationalmap.gov/v1/json?'

def elevation_function(df, lat_column, lon_column):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):
                
        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Feet'
        }
        
        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        print(result.text)  # print the response text
        try:
            elevations.append(result.json()['value'])
        except json.JSONDecodeError:
            print(f"Failed to decode JSON for {lat}, {lon}")
            elevations.append(None)

    df['altitude'] = elevations

elevation_function(df, 'latitude', 'longitude')
df.head()

{"location":{"x":-80.015833,"y":40.446667,"spatialReference":{"wkid":4326,"latestWkid":4326}},"locationId":0,"value":723.8302187137473,"rasterId":77932,"resolution":1}
{"location":{"x":-98.478889,"y":29.416944,"spatialReference":{"wkid":4326,"latestWkid":4326}},"locationId":0,"value":661.6667210814709,"rasterId":64457,"resolution":1}
{"location":{"x":-115.183889,"y":36.090556,"spatialReference":{"wkid":4326,"latestWkid":4326}},"locationId":0,"value":2186.1285811601806,"rasterId":14311,"resolution":1}
Invalid or missing input parameters.
Failed to decode JSON for 48.218889, 11.624722
{"location":{"x":-97.092778,"y":32.747778,"spatialReference":{"wkid":4326,"latestWkid":4326}},"locationId":0,"value":566.7295161701209,"rasterId":75501,"resolution":1}
{"location":{"x":-80.852778,"y":35.225833,"spatialReference":{"wkid":4326,"latestWkid":4326}},"locationId":0,"value":707.1364621559251,"rasterId":13146,"resolution":0.00003086419871794868}
{"location":{"x":-78.787,"y":42.7738,"spatialReferenc

,name,domed,retractable,latitude,longitude,altitude
0,Acrisure Stadium,0,NaN,40.446667,-80.015833,723.830219
1,Alamodome,1,0.0,29.416944,-98.478889,661.666721
2,Allegiant Stadium,1,0.0,36.090556,-115.183889,2186.128581
3,Allianz Arena,0,NaN,48.218889,11.624722,NaN
4,AT&T Stadium,1,1.0,32.747778,-97.092778,566.729516


In [27]:
for i,row in df.iterrows():
    altitude = row['altitude'] if pd.notna(row['altitude']) else None
    name = row['name'] if pd.notna(row['name']) else None
    sql = "UPDATE stadiums SET altitude = %s WHERE name = %s"
    data = (altitude, name)
    cursor.execute(sql,data)

mydb.commit()

if (mydb.is_connected()):
    cursor.close()
    mydb.close()
    print("MySQL connection is closed")

MySQL connection is closed
